In [2]:
# BLOK 1

# importovanie potrebnych kniznic

from keras.models import Sequential
from keras.layers import Convolution1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.utils import np_utils
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import json
import pickle
np.random.seed(1234)

Using TensorFlow backend.


In [37]:
data = pd.read_csv('df_nospotty_all.csv',
                   header=0,
                   nrows=10
                   #usecols=['morphology', 'generic_bessell_v']
                   )
print(data)


   Unnamed: 0  id    morphology  \
0           2   3  over-contact   
1           3   4  over-contact   
2           5   6  over-contact   
3           7   8  over-contact   
4           8   9  over-contact   
5          10  11  over-contact   
6          12  13  over-contact   
7          14  15  over-contact   
8          16  17  over-contact   
9          18  19  over-contact   

                                              params  \
0  {"system": {"inclination": 35.0, "period": 0.2...   
1  {"system": {"inclination": 35.0, "period": 0.3...   
2  {"system": {"inclination": 35.0, "period": 0.4...   
3  {"system": {"inclination": 35.0, "period": 0.5...   
4  {"system": {"inclination": 35.0, "period": 0.6...   
5  {"system": {"inclination": 35.0, "period": 0.7...   
6  {"system": {"inclination": 35.0, "period": 0.8...   
7  {"system": {"inclination": 40.0, "period": 0.2...   
8  {"system": {"inclination": 40.0, "period": 0.3...   
9  {"system": {"inclination": 40.0, "period": 0.4...  

In [4]:
for row in data['generic_bessell_v']:
    row = json.loads(row)

print(data['generic_bessell_v'][0])


[1.0057136315963416e+24, 9.95135723875756e+23, 9.851943559827183e+23, 9.762193458186489e+23, 9.683913031613861e+23, 9.618904977996711e+23, 9.570628633797492e+23, 9.540090181588095e+23, 9.527041421357296e+23, 9.531458653481288e+23, 9.553106151988257e+23, 9.592988744435728e+23, 9.649098052815326e+23, 9.721533483027615e+23, 9.806023172178183e+23, 9.900547836840691e+23, 1.0003535898779049e+24, 1.0112602961872905e+24, 1.0226315660435644e+24, 1.0340621235216665e+24, 1.0454618349338969e+24, 1.0563404430623323e+24, 1.0666205437602627e+24, 1.076093908515621e+24, 1.0843070105694854e+24, 1.0912001754747813e+24, 1.0966160174070761e+24, 1.1004891716056055e+24, 1.1026990857612189e+24, 1.1030915105362091e+24, 1.1017035459444325e+24, 1.0985290645060521e+24, 1.093751266025706e+24, 1.0874299305395066e+24, 1.0796959637690447e+24, 1.0706519220881196e+24, 1.0604374351529858e+24, 1.049561040672905e+24, 1.0380782749529487e+24, 1.0262235063153202e+24, 1.0144068105644953e+24, 1.0027506437794815e+24, 9.91692295

KeyError: 0

In [18]:
newData = []
for row in data['generic_bessell_v']:
    row = json.loads(row)
    minValueOfRow, maxValueOfRow = min(row), max(row)
    newRow = []
    for valueIndex in range(len(row)):
        row[valueIndex] = (row[valueIndex] - minValueOfRow) / (maxValueOfRow - minValueOfRow)
        newRow.append([row[valueIndex]])
    newData.append(newRow)
    
newData = np.array(newData)



In [33]:
target = []
oc, sd, dt = 0, 0, 0
for row in data['morphology']:
    if row == 'over-contact':
        target.append(0)
        oc += 1
    # elif row == 'semi-detached':
    #     target.append(1)
    #     sd += 1
    elif row == 'detached':
        target.append(1)
        dt += 1
        
target = np.array(target)
print("Over-contact: " + str(oc) + "\n" + "Semi-detached: " + str(sd) + "\n" + "Detached: " + str(dt))


Over-contact: 45869
Semi-detached: 0
Detached: 445556


In [35]:
from random import randint

OCcurves, DTcurves = [], []
for i in range(len(target)):
    if target[i] == 0:
        OCcurves.append(newData[i])
    elif target[i] == 1:
        DTcurves.append(newData[i])
        
print(len(OCcurves))
print(len(DTcurves))

for i in range(10000):
    rnd = randint(0, 45868)
    plt.plot(OCcurves[rnd])
plt.title('Over-contact')
plt.show()

for i in range(10000):
    rnd = randint(0, 445555)
    plt.plot(DTcurves[rnd])
plt.title('Detached')
plt.show()  
    



45869
445556


In [19]:
print("Pocet kriviek: " + str(len(newData)) + "\nPocet cieloveho atributu: " + str(len(target)))
X_train, X_test, y_train, y_test = train_test_split(newData, target, test_size=0.2)
y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)
print(len(X_train))
print(len(X_test))



Pocet kriviek: 491425
Pocet cieloveho atributu: 491425


393140
98285


In [23]:
# architektura a trenovanie MODEL 3

classifier = Sequential()
classifier.add(Convolution1D(64, 20, activation='relu', input_shape=(100, 1)))
classifier.add(MaxPooling1D(pool_size=2))
classifier.add(Convolution1D(32, 10, activation='relu'))
classifier.add(MaxPooling1D(pool_size=2))
classifier.add(Flatten())
classifier.add(Dense(64, activation='relu'))
classifier.add(Dropout(0.25))
classifier.add(Dense(2, activation='softmax'))
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# history = classifier.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=32, verbose=1)
# accuracy = classifier.evaluate(X_test, y_test, batch_size=32, verbose=1)
# print("Celková presnosť modelu 3: " + str(accuracy))


In [24]:
history = classifier.fit(X_train, y_train, validation_split=0.2, epochs=5, batch_size=32, verbose=1)
accuracy = classifier.evaluate(X_test, y_test, batch_size=32, verbose=1)
print("Celková presnosť modelu 3: " + str(accuracy))


Train on 314512 samples, validate on 78628 samples
Epoch 1/5


    32/314512 [..............................] - ETA: 1:16:11 - loss: 0.7040 - accuracy: 0.5000

   160/314512 [..............................] - ETA: 17:22 - loss: 0.4633 - accuracy: 0.8375  

   288/314512 [..............................] - ETA: 10:42 - loss: 0.4205 - accuracy: 0.8507

   416/314512 [..............................] - ETA: 8:07 - loss: 0.3609 - accuracy: 0.8750 

...

96704/98285 [============================>.] - ETA: 0s

97408/98285 [============================>.] - ETA: 0s

98144/98285 [============================>.] - ETA: 0s

98285/98285 [==============================] - 8s 77us/step


Celková presnosť modelu 3: [0.001487313110835089, 0.9996032118797302]


In [25]:
classifier.save("NEW_Model3.h5")


In [28]:
# vyhodnotenie modelu na syntetickych datach pomocou kontingencnej tabulky
# vypocet presnosti, navratnosti a f1 skore

y_pred = classifier.predict(X_test)
y_pred2 = []
for i in y_pred:
    maximum = np.argmax(i)
    y_pred2 = np.append(y_pred2, maximum)
y_pred2 = np_utils.to_categorical(y_pred2, 2)

cm = confusion_matrix(y_test.argmax(axis=1), y_pred2.argmax(axis=1))
print("Kontingencna tablulka: \n" + str(cm))
prfs = precision_recall_fscore_support(y_test.argmax(axis=1), y_pred2.argmax(axis=1), average=None)
print("Presnost, navratnost, fmiera, support: " + str(prfs))


Kontingencna tablulka: 
[[ 9233     4]
 [   35 89013]]
Presnost, navratnost, fmiera, support: (array([0.99622356, 0.99995506]), array([0.99956696, 0.99960695]), array([0.99789246, 0.99978098]), array([ 9237, 89048]))


In [ ]:
########################################################

# OBSERVACNE DATA

########################################################


In [86]:
# nacitanie observacnych dat a cieloveho atributu
# vyber atributu 'flux', ktory predstavuje svietivost svetelnych kriviek a ich transformacia na numpy polia
# preskalovanie hodnot svetelnej krivky podla MinMax skalovania
# transformacia oznacenia cieloveho atributu: over-contact - 0, semi-detached - 1, detached - 2

processed = pd.read_pickle('observation_data.pkl')['processed_lightcurve']
morphology = pd.read_pickle('observation_data.pkl')['morphology']
curves = []
for j in processed:
    processed_data = eval(j)
    processed_data = np.array(processed_data['flux'])
    single_curve = []
    for i in range(len(processed_data)):
        point = []
        point_data = (processed_data[i] - processed_data.min()) / (processed_data.max() - processed_data.min())
        point.extend([point_data])
        single_curve.append(point)
    curves.append(single_curve)
    single_curve = pd.DataFrame(single_curve)
curves = np.array(curves)

target = []
for i in morphology:
    if i == 'over-contact':
        target.extend([0])
    if i == 'semi-detached':
        target.extend([1])
    if i == 'detached':
        target.extend([1])
target = np.array(target)

# odstranenie semi-detached kriviek
# filtered_curves = []
# for i in range(len(morphology)):
#     if morphology[i] != 'semi-detached':
#         filtered_curves.append(curves[i])
# curves = filtered_curves
###################################

target = np_utils.to_categorical(target, 2)


In [87]:
newObservation = []
for curve in curves:
    newCurve = []
    for valueIndex in range(len(curve)):
        if valueIndex % 2 == 1:
            newCurve.append(curve[valueIndex])
    newCurve = np.array(newCurve)
    newObservation.append(newCurve)
newObservation = np.array(newObservation)

print(len(newObservation))
print(len(newObservation[0]))


100
100


In [85]:
for i in newObservation:
    plt.plot(i)
plt.title('New observation')
plt.show()

for i in curves:
    plt.plot(i)
plt.title('Origin observation')
plt.show()



In [88]:
y_pred = classifier.predict(newObservation)
y_pred2 = []
for i in y_pred:
    maximum = np.argmax(i)
    y_pred2 = np.append(y_pred2, maximum)
# plt.bar(i, height=1)
# plt.show()
y_pred2 = np_utils.to_categorical(y_pred2, 2)
cm = confusion_matrix(target.argmax(axis=1), y_pred2.argmax(axis=1))
print("Kontingencna tablulka: \n" + str(cm))
prfs = precision_recall_fscore_support(target.argmax(axis=1), y_pred2.argmax(axis=1), average=None)
print("Presnost, navratnost, fmiera, support: " + str(prfs))


Kontingencna tablulka: 
[[41  6]
 [12 41]]
Presnost, navratnost, fmiera, support: (array([0.77358491, 0.87234043]), array([0.87234043, 0.77358491]), array([0.82, 0.82]), array([47, 53]))
